# Comparison of aviation climate models

In [1]:
import math
import numpy as np
import pandas as pd
import time
from pandas import read_csv
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from aerometrics.climate_models.fair_climate_model import background_species_quantities_function, species_fair_climate_model
from aerometrics.climate_models.climate_models import aviation_climate_model

# Sensitivity analysis
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib import ProblemSpec

## Parameters

In [2]:
scenario = "Historical"

In [3]:
commercial_aviation_historical_data_df = read_csv("../climate_data/aviation_emissions_data.csv", delimiter=";")
commercial_aviation_historical_data = commercial_aviation_historical_data_df.values

if scenario == "Historical":
    years_data = commercial_aviation_historical_data[:80, 0]
    co2_emissions_data = commercial_aviation_historical_data[:80, 1]
    nox_emissions_data = commercial_aviation_historical_data[:80, 2]
    h2o_emissions_data = commercial_aviation_historical_data[:80, 3]
    soot_emissions_data = commercial_aviation_historical_data[:80, 4]
    sulfur_emissions_data = commercial_aviation_historical_data[:80, 5]
    distance_data = commercial_aviation_historical_data[:80, 6]
    
else:
    years_data = commercial_aviation_historical_data[:, 0]
    co2_emissions_data = commercial_aviation_historical_data[:, 1]
    nox_emissions_data = commercial_aviation_historical_data[:, 2]
    h2o_emissions_data = commercial_aviation_historical_data[:, 3]
    soot_emissions_data = commercial_aviation_historical_data[:, 4]
    sulfur_emissions_data = commercial_aviation_historical_data[:, 5]
    distance_data = commercial_aviation_historical_data[:, 6]

species_quantities = np.zeros((7, len(years_data)))

species_quantities[0] = co2_emissions_data * 1e9 # [Mt to kg]
species_quantities[1] = distance_data
species_quantities[2] = nox_emissions_data * 1e9 # [Mt to kg]
species_quantities[3] = nox_emissions_data * 1e9 # [Mt to kg]
species_quantities[4] = h2o_emissions_data * 1e9 # [Mt to kg]
species_quantities[5] = soot_emissions_data * 1e9 # [Mt to kg]
species_quantities[6] = sulfur_emissions_data * 1e9 # [Mt to kg]

if scenario == "Trend":
    pass
elif scenario == "Stabilisation":
    for i in range(0,7):
        for j in range(110, len(years_data)):
            species_quantities[i, j] = species_quantities[i, j-1] 
            
elif scenario == "Halt":
    for i in range(0,7):
        for j in range(110, len(years_data)):
            species_quantities[i, j] = 0  

start_year = int(years_data[0])
end_year = int(years_data[-1])

In [4]:
background_species_quantities = background_species_quantities_function(start_year, end_year, rcp='RCP45')
sensitivity_rf_gwpstar = [0, 2.23e-12, 25.1e-12 * (14/46), -26.1e-12 * (14/46) * 0.77, 0.0052e-12, 100.7e-12, -19.9e-12]
sensitivity_rf_others = [0, 2.23e-12, 25.1e-12 * (14/46), -0.83e-9, 0.0052e-12, 100.7e-12, -19.9e-12]
ratio_erf_rf = [1, 0.42, 1.37, 1.18, 1, 1, 1]
efficacy_erf = [1, 1, 1, 1, 1, 1, 1]
tcre = 0.00045

species_settings_gwpstar = {"sensitivity_rf": sensitivity_rf_gwpstar, "ratio_erf_rf": ratio_erf_rf, "efficacy_erf": efficacy_erf}
species_settings_others = {"sensitivity_rf": sensitivity_rf_others, "ratio_erf_rf": ratio_erf_rf, "efficacy_erf": efficacy_erf}

model_settings_others = {"tcre": tcre}
model_settings_fair = {"background_species_quantities": background_species_quantities}

## Calculation

In [5]:
# Define problem
sp = ProblemSpec({
    'names': ['sensitivity_rf', 'ratio_erf_rf', 'efficacy_erf'],
    'bounds': [ 
        [2.23e-12, 7.96e-13],
        [0.23, 0.87], 
        [0.12, 0.64], 
    ],
    'dists': ['norm', 'unif', 'unif'],
})

In [ ]:
# Create wrapper for function

def wrapped_model(params):
    n_samples = params.shape[0]
    n_times = end_year - start_year + 1
    results = np.empty((n_samples, n_times))

    for i in range(n_samples):
        sensitivity_rf_others = [0, params[i,0], 25.1e-12 * (14/46), -0.83e-9, 0.0052e-12, 100.7e-12, -19.9e-12]
        ratio_erf_rf = [1, params[i,1], 1.37, 1.18, 1, 1, 1]
        efficacy_erf = [1, params[i,2], 1, 1, 1, 1, 1]
        species_settings_others = {"sensitivity_rf": sensitivity_rf_others, "ratio_erf_rf": ratio_erf_rf, "efficacy_erf": efficacy_erf}
        results[i, :] = aviation_climate_model(start_year, end_year, "FaIR", species_quantities, species_settings_others, model_settings_fair)[5].flatten()
    
    return results

y_results = np.zeros((9, 2))
# Run all steps
for k in range(3,12):
    nb_samples = 2**k 
    (
        sp.sample_sobol(nb_samples, calc_second_order=False)  
        .evaluate(wrapped_model)
        .analyze_sobol()
    )

    S1s = np.array([sp.analysis[_y]['S1'] for _y in sp['outputs']])
    y = sp.results

    y_results[k-3, 0] = 1000* np.mean(y, axis=0)[-1]
    y_results[k-3, 1] = 1000* np.std(y, axis=0)[-1]
    
    print(y_results)

C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\SALib\analyze\sobol.py:141: RuntimeWarning: invalid value encountered in divide
  Y = (Y - Y.mean()) / Y.std()


[[8.46666371 5.09503086]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]]


C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\SALib\analyze\sobol.py:141: RuntimeWarning: invalid value encountered in divide
  Y = (Y - Y.mean()) / Y.std()


[[8.46666371 5.09503086]
 [8.52442939 5.55359076]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]]


C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\SALib\analyze\sobol.py:141: RuntimeWarning: invalid value encountered in divide
  Y = (Y - Y.mean()) / Y.std()


...

[[8.46666371 5.09503086]
 [8.52442939 5.55359076]
 [8.57989524 5.57739563]
 [8.74932926 5.81331286]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]]


C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\SALib\analyze\sobol.py:141: RuntimeWarning: invalid value encountered in divide
  Y = (Y - Y.mean()) / Y.std()


[[8.46666371 5.09503086]
 [8.52442939 5.55359076]
 [8.57989524 5.57739563]
 [8.74932926 5.81331286]
 [8.75484946 5.80996909]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]]


C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
C:\Users\t.planes\AppData\Local\miniconda3\envs\AeroMetrics\lib\site-packages\SALib\analyze\sobol.py:141: RuntimeWarning: invalid value encountered in divide
  Y = (Y - Y.mean()) / Y.std()


[[8.46666371 5.09503086]
 [8.52442939 5.55359076]
 [8.57989524 5.57739563]
 [8.74932926 5.81331286]
 [8.75484946 5.80996909]
 [8.75881814 5.87410311]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]]


In [ ]:
# Get first order sensitivities for all outputs
S1s = np.array([sp.analysis[_y]['S1'] for _y in sp['outputs']])

# Get model outputs (each output is a given time horizon)
y = sp.results

# Set up figure
fig, ax0 = plt.subplots(figsize=(6, 4))

ax0.plot(years_data, 1000* np.mean(y, axis=0), label="Mean", color='black', linestyle='--')

ax0.plot(years_data, 1000* np.median(y, axis=0), label="Median", color='black')

# in percent
prediction_interval = 95

ax0.fill_between(years_data,
                 1000* np.percentile(y, 50 - prediction_interval/2., axis=0),
                 1000* np.percentile(y, 50 + prediction_interval/2., axis=0),
                 alpha=0.2, color='black',
                 label=f"{prediction_interval} % prediction interval")

prediction_interval = 50
ax0.fill_between(years_data,
                 1000* np.percentile(y, 50 - prediction_interval/2., axis=0),
                 1000* np.percentile(y, 50 + prediction_interval/2., axis=0),
                 alpha=0.3, color='black',
                 label=f"{prediction_interval} % prediction interval")

ax0.plot(years_data, 1000* fair_climate_model[5], label="Lee et al. settings", color='blue')

ax0.set_xlabel("Year")
ax0.set_ylabel("Surface temperature change [mK]")
ax0.set_title("Contrail climate impact")
ax0.legend(loc='upper left')
ax0.grid(True)
ax0.set_xlim(1940, 2019)

plt.savefig("test1.pdf")
plt.show()

In [ ]:
x = [2**i for i in range(3,12)]

plt.figure(figsize=(8, 5))
plt.plot(x, y_results[:,0]/y_results[-1,0], marker='o', label="Mean")
plt.plot(x, y_results[:,1]/y_results[-1,1], marker='o', label="Standard deviation")
plt.fill_between(x, 0.95*np.ones(len(y_results[:,0])), 1.05*np.ones(len(y_results[:,0])), alpha = 0.5, color='grey', label="±5% range")
plt.xscale('log', base=2)
plt.legend()

plt.xlabel('Number of samples')
plt.ylabel('Indicators (normalised on last simulation)')
plt.grid(True)

plt.show()